In [2]:
#import pynq libraries
from pynq import Overlay
from pynq import allocate
import math
import time
import numpy as np

In [3]:
#show documentation
#testmem_ip?

In [4]:
#show testmem ip register maps
#testmem_ip.register_map

In [15]:
Mbyte=20
for n in range(3):
    #kernel function's IP made for write from the FPGA
    if n==0:
        overlay =Overlay("./design_64_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=64
        w_time_64= np.zeros(20)
    if n==1:
        overlay =Overlay("./design_1_128_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=128
        w_time_128=np.zeros(20)
    if n==2:
        overlay =Overlay("./design_1_256_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=256
        w_time_256=np.zeros(20)
    
    #inizialize data
    max_data=int((Mbyte*1000*1000*8)/datatype)
    #FPGA
    data= np.zeros(int(max_data*datatype/64))
    #data buffer creation for FPGA
    buf_data = allocate(int(max_data*datatype/64),np.uint64)
    buf_data[:]=data[:]
    buf_data.flush()
    testmem_ip.write(0x10,buf_data.physical_address)
    
    #kernel function attivation and time calculation
    
    #decide Mbyte intervals 
    sud=int(Mbyte/20)
    data_n=int((sud*1000*1000*8)/datatype)
    #starting write time calculation
    
    for p in range(20):
        p=1+p
        #100 subtests to create an avarage time
        w_sum_64=0
        w_sum_128=0
        w_sum_256=0
        for c in range(100):
            data_n=int((sud*1000*1000*8)/datatype) #you can choose 1024 insted of 1000 if you use other 2^n data
            data_n=data_n*(p)
            testmem_ip.write(0x1c,data_n)
            testmem_ip.write(0x00,1)
            #start counting
            start=time.time()
            #continue until it end 
            while testmem_ip.read(0x00)& 0x04!=0x04:
                pass
            #finish counting
            stop=time.time()
            if n==0:
                w_sum_64=w_sum_64+(stop-start)*1000
            if n==1:
                w_sum_128=w_sum_128+(stop-start)*1000
            if n==2:
                w_sum_256=w_sum_256+(stop-start)*1000
        if n==0:
            w_time_64[p-1]=w_sum_64/100
        if n==1:
            w_time_128[p-1]=w_sum_128/100
        if n==2:        
            w_time_256[p-1]=w_sum_256/100
    print(data_n)

2500000
1250000
625000


In [31]:
print(w_time_64)
print(w_time_128)
print(w_time_256)


[ 1.73228502  3.45255852  5.18655062  6.89007282  8.60830784 10.32785654
 12.04707861 13.7644124  15.48511982 17.20361948 18.92184258 20.6390667
 22.35848188 24.07720327 25.79684734 27.51544476 29.23327684 30.95252275
 32.67133951 34.39054012]
[ 1.57544613  3.13866377  4.70213413  6.26619101  7.82869101  9.39023256
 10.95297575 12.57017612 14.07886505 15.64100981 17.20303535 18.76620531
 20.32797337 21.89190149 23.45444441 25.01683235 26.58080816 28.14123631
 29.7278285  31.26753807]
[ 1.41936541  2.82815695  4.23460722  5.64878702  7.0476222   8.45356941
  9.85909939 11.26732349 12.67540455 14.07867432 15.48501015 16.89722538
 18.2972002  19.70334291 21.11042976 22.51682758 23.92241716 25.32778978
 26.7341876  28.14126968]


In [51]:
bit=1000
for n in range(3):
    #kernel function's IP made for write from the FPGA
    if n==0:
        overlay =Overlay("./design_64_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=64
        w_time1_64= np.zeros(20)
    if n==1:
        overlay =Overlay("./design_1_128_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=128
        w_time1_128=np.zeros(20)
    if n==2:
        overlay =Overlay("./design_1_256_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=256
        w_time1_256=np.zeros(20)
    
    #inizialize data
    max_data=int(bit/datatype)
    #FPGA
    data= np.zeros(int(max_data*datatype/64))
    #data buffer creation for FPGA
    buf_data = allocate(int(max_data*datatype/64),np.uint64)
    buf_data[:]=data[:]
    buf_data.flush()
    testmem_ip.write(0x10,buf_data.physical_address)
    
    #kernel function attivation and time calculation
    
    data_n=int(bit/datatype)
    #starting write time calculation
    
    #100 subtests to create an avarage time
    w_sum_64=0
    w_sum_128=0
    w_sum_256=0
    for c in range(1000):
        data_n=int(bit/datatype)
        testmem_ip.write(0x1c,data_n)
        testmem_ip.write(0x00,1)
        #start counting
        start=time.time()
        #continue until it end 
        while testmem_ip.read(0x00)& 0x04!=0x04:
            pass
        #finish counting
        stop=time.time()
        if n==0:
            w_sum_64=w_sum_64+(stop-start)*1000
        if n==1:
            w_sum_128=w_sum_128+(stop-start)*1000
        if n==2:
            w_sum_256=w_sum_256+(stop-start)*1000
    if n==0:
        w_time1_64=w_sum_64/1000
    if n==1:
        w_time1_128=w_sum_128/1000
    if n==2:        
         w_time1_256=w_sum_256/1000
    print(data_n)

15
7
3


In [52]:
print( w_time1_64)
print(w_time1_128)
print( w_time1_256)

0.027930498123168945
0.0276334285736084
0.028670310974121094
